## ML101 : Inside the decision tree classifier
10-08-2016  
Jan Fait, jan.fait@teradata.com  
Digital Marketing  
Munich  


## 1. Getting ready

+ You may get your Python IDE ready if interested
+ This tutorial draws heavily from Programing Collective Intelligence, Chapter 7, Decision Trees

--------------------------------------------------------------------------------------------------------------

### Decision trees are a supervised (machine)-learning technique

+ Supervised learning models classify new data based on previously available data.  
+ They know what combination of inputs led to what outcomes. 
+ Tree classifies a new data point based on a series of conditional statements. 

------------------------


--------------------------------------------------------------------------------------------------------------
### Examples of what decision trees commonly solve

* Fraud detection  
* Customer/Item classification based on usage, properites
* Conversion prediction

--------------------------------------------------------------------------------------------------------------

------------------------

### Decision trees are good because


 + Easily understood, you see how stuff happens
 + No parametric demands and assumptions about distributions
 + Automated feature selection and variable importance
 + Little pre-processing of data needed, easy missing value treatment
 + Both categorical and continuous predictors

**Quick to deploy, good start, everybody knows what's goin on**
 


------------------------
### Decision trees are bad because

 - Very eager to overfit
 - Correlation between inputs confuses the split algorithm
 
 
**Check for too specific, arbitrary split rules that likely don't apply outside of the input sample**

--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------
### A beer tree
 
+ Input = [brand,price]
+ Output = [purchase decision]

In [ ]:
beer_data=[['paulaner',2,'no'],
           ['paulaner',3,'no'],
           ['augustiner',3,'no'],
           ['augustiner',3,'yes'],
           ['paulaner',0,'no'], #oh, look at this, so true
           ['augustiner',3,'yes'],
           ['augustiner',3,'yes'],
           ['augustiner',3,'no'],
           ['paulaner',2,'no'],
           ['paulaner',3,'no']]

![Paulaner-sucks tree](/files/tree.jpg)


Easy, but how **exactly** did the tree do it? 

>How does it know to put the brand as the first node?

>What are we trying to achieve on the end nodes?

--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------
### The CART algorithm
(classification-and-regression-trees algorithm, Breiman, 1984)  
See Introduction for Statistical Learning, p.303 for regression trees


> The algorithm relies on recursive-partioning of the dataset by maximizing the **purity** of the outcome columns in the resulting sets.

Core question CART asks itself : 
> Does this split lead to a more pure node with respect to the outcome column?

### The CART algorithm
highest_purity=purity(data)  
purity_improvement=0  
best_split = []  

For each column in data
....For each value in unique values of column

.........A = data[column==value]   
.........B = data[column!=value]  
.........Compute purity[value,column] = len(A)/len(data) purity(A) + len(B)/len(data) purity(B).  
.........If purity[value,column] < highest_purity  
............purity_improvement = highest_purity - purity[value,column]  
............highest_purity = purity[value,column]  
............best_split = [A,B]  
return best_split  
...If purity_improvement > 0  
......For each subset of best_split  
.........For each column in subset ... and so on

--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------
### How to measure purity of a split

Every time the CART algorithm proposes a split on a value, we check how pure the 2 resulting sets are with respect to the outcome variable.

The most common methods (for classification) are:

###### Entropy

The amount of disorder in a list.


In [12]:
less_mixed_set = ["cat","cat","cat","cat","cat","dog"]
more_mixed_set = ["cat","cat","cat","dog","dog","dog"]

# @x = list of values 
# get unique values of the list, for each of them check how many there are as a share of list length (p)
# multiply the p by its log(), add up the products.
# (as the p is (0-1) logs are 0< by definition, to add up negatives, we do -= decrement)

def entropy(x):
   from math import log
   log2=lambda y:log(y)/log(2)
   uniquevals=list(set(x))
   ent = 0.0
   for val in uniquevals:
    p = len([item for item in x if item==val])/len(x)
    ent -= p*log2(p)
   return ent

entropy(less_mixed_set), entropy(more_mixed_set)

(0.6500224216483541, 1.0)

###### Gini Impurity
It is the theorerical probability of misclassifying when taking a random item from a set and assigning a random label to it.

In [13]:
# @x = list of values
#get unique values of the list, for each of them check how many there are as a share of list length (p1)
#multiply this p1 by a probabilities of other items in the list
#add up probabilities

def gini(x):
   uniquevals=list(set(x))
   g = 0.0
   for val1 in uniquevals:
    p1 = len([item for item in x if item==val1])/len(x)
    for val2 in uniquevals:
        if val1==val2: continue
        p2 = len([item for item in x if item==val2])/len(x)
        g += p1*p2
   return g

gini(less_mixed_set),gini(more_mixed_set)

(0.2777777777777778, 0.5)

--------------------------------------------------------------------------------------------------------------

###### Information gain

For the next steps, we chose **entropy** as it is stricter. Look at the less_mixed_set, just 1 dog among 5 cats brings entropy to 0.65. Which is actually quite accurate to real-life.

Entropy is our purity measure, but **Information gain** is actually what decides which split to take first. See below how to get it. It is the difference between current node and potential split node entropies. The split that has the highest information gain is taken.

![Information gain with entropy](/files/information_gain.jpg)
--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------

## 2. Structure of our program


### What do we need

1. function that does the looping through input columns and their values and divides data into sets
2. function that computes the purity of the divided sets
3. function that tells us if the purity is pure enough to do the split
4. way to create a recursive application of the above
5. function that displays the tree

### Program in pseudo-code

In [ ]:
 function build_decision_tree (data){
        
        old_entropy = entropy(data)
        best_information_gain = 0
        minimal_gain_needed = 0
        best_splits = null
        
            for each column of data:
                for each value of column
                    splits[] = **divide(data,value,column)** (1.)
                    new_entropy = **entropy(splits)** (2.)
                    new_information_gain = old_entropy - new_entropy
                    
                    if **new_information_gain > best_information_gain** (3.)
                        best_information_gain = new_information_gain
                        best_splits = splits
        
            if **best_information_gain > minimal_gain_needed**
                **branch0 = build_decision_tree(splits[0])** (4.)
                branch1 = build_decision_tree(splits[1])
                return split_node 
            else
                return end_node
             
}

--------------------------------------------------------------------------------------------------------------
### Program in Python

In [12]:
# 1. Divide function
def divideset(rows,column,value):
   split_function=None
   if isinstance(value,int) or isinstance(value,float):
      split_function=lambda row:row[column]>=value
   else:
      split_function=lambda row:row[column]==value
   set1=[row for row in rows if split_function(row)]
   set2=[row for row in rows if not split_function(row)]
   return (set1,set2)

# Helper function to create counts of possible results (the last column of each row is the result)
def uniquecounts(rows):
   results={}
   for row in rows:
      # The result is the last column
      r=row[len(row)-1]
      if r not in results: results[r]=0
      results[r]+=1
   return results

#  2. Edited entropy to be applied in this scenario
def entropy(rows):
   from math import log
   log2=lambda x:log(x)/log(2)  
   results=uniquecounts(rows)
   # Now calculate the entropy
   ent=0.0
   for r in results.keys():
      p=float(results[r])/len(rows)
      ent=ent-p*log2(p)
   return ent

# Helper class to build the decision node that is meant to be returned
class decisionnode:
  def __init__(self,col=-1,value=None,results=None,tb=None,fb=None):
    self.col=col
    self.value=value
    self.results=results
    self.tb=tb
    self.fb=fb

And now on to the actual program as outlined in pseudo-code above. Notice we pass the entropy function as a purity measure. This setup allows us to pass gini or other if need be.

In [98]:
def buildtree(rows,scoref=entropy):
  if len(rows)==0: return decisionnode()
  current_score=scoref(rows)

  best_gain=0.0
  best_criteria=None
  best_sets=None
  
  column_count=len(rows[0])-1
  for col in range(0,column_count):
    column_values={}
    for row in rows:
       column_values[row[col]]=1
    for value in column_values.keys():
      (set1,set2)=divideset(rows,col,value)
      
      # Information gain
      p=float(len(set1))/len(rows)
      gain=current_score-p*scoref(set1)-(1-p)*scoref(set2)
      if gain>best_gain and len(set1)>0 and len(set2)>0:
        best_gain=gain
        best_criteria=(col,value)
        best_sets=(set1,set2)
  # Create the sub branches   
  if best_gain>0:
    trueBranch=buildtree(best_sets[0])
    falseBranch=buildtree(best_sets[1])
    return decisionnode(col=best_criteria[0],value=best_criteria[1],
                        tb=trueBranch,fb=falseBranch)
  else:
      return decisionnode(results=uniquecounts(rows))

--------------------------------------------------------------------------------------------------------------
### Running the program on data

Take beer data and extend them with another variable of whether the beer is light (helles) or dark (dunkles).

In [99]:
beer_data2=[['paulaner',2,'light','yes'],
           ['paulaner',3,'dark','no'],
           ['augustiner',5,'dark','no'],
           ['augustiner',5,'light','no'],
           ['augustiner',5,'light','no'],
           ['paulaner',2,'light','yes'],
           ['augustiner',3,'light','yes'],
           ['augustiner',3,'light','yes'],
           ['augustiner',5,'dark','no'],
           ['paulaner',2,'light','yes'],
           ['paulaner',3,'light','no'],
           ['paulaner',3,'light','no'],
           ['paulaner',3,'dark','no'],
           ['paulaner',3,'dark','no'],
           ['augustiner',4,'light','no'],
           ['paulaner',2,'dark','yes'],
           ['paulaner',2,'dark','yes'],
           ['augustiner',4,'light','no'],
           ['augustiner',4,'dark','yes'],
           ['augustiner',4,'dark','yes'],
           ['augustiner',4,'dark','yes'],
           ['augustiner',4,'dark','yes'],
           ['augustiner',4,'dark','yes'],
           ['augustiner',4,'light','no'],
           ['tegnerseer',7,'dark','yes'],#look at these outliers
           ['tegnerseer',0,'dark','no']]#look at these outliers


beer_tree = buildtree(beer_data2)

--------------------------------------------------------------------------------------------------------------
### Display function

Just for completeness

In [ ]:
def getwidth(tree):
  if tree.tb==None and tree.fb==None: return 1
  return getwidth(tree.tb)+getwidth(tree.fb)

def getdepth(tree):
  if tree.tb==None and tree.fb==None: return 0
  return max(getdepth(tree.tb),getdepth(tree.fb))+1


from PIL import Image,ImageDraw,ImageFont

def drawtree(tree,jpeg='tree.jpg'):
  w=getwidth(tree)*100
  h=getdepth(tree)*100+120

  img=Image.new('RGB',(w,h),(255,255,255))
  draw=ImageDraw.Draw(img)

  drawnode(draw,tree,w/2,20)
  img.save(jpeg,'JPEG')
  
def drawnode(draw,tree,x,y):
  if tree.results==None:
    # Get the width of each branch
    w1=getwidth(tree.fb)*100
    w2=getwidth(tree.tb)*100

    # Determine the total space required by this node
    left=x-(w1+w2)/2
    right=x+(w1+w2)/2

    # Draw the condition string
    draw.text((x-20,y-10),str(tree.col)+':'+str(tree.value),(0,0,0))

    # Draw links to the branches
    draw.line((x,y,left+w1/2,y+100),fill=(255,0,0))
    draw.line((x,y,right-w2/2,y+100),fill=(255,0,0))
    
    # Draw the branch nodes
    drawnode(draw,tree.fb,left+w1/2,y+100)
    drawnode(draw,tree.tb,right-w2/2,y+100)
  else:
    txt=' \n'.join(['%s:%d'%v for v in tree.results.items()])
    draw.text((x-20,y),txt,(0,0,0))

--------------------------------------------------------------------------------------------------------------
### Looking at the tree

Our algorithm produces the below tree.

In [101]:
drawtree(beer_tree,"beer_tree.jpg")

![Information gain with entropy](/files/beer_tree.jpg)

--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------
## 3. Pruning tree and classifying new data


### Overfitting

Well, our tree **overfits**. See that single "tegnerseer" in its own branch?. It messes up the tree. Once we collect more tegnerseers, we will surely place them somewhere else in the tree. Overfitting means following the data too closely.

Making predictions more generally applicable means reducing the tree's complexity --> **Pruning the tree**.

In [108]:
def prune(tree,mingain):
  # If the branches aren't leaves, then prune them
  if tree.tb.results==None:
    prune(tree.tb,mingain)
  if tree.fb.results==None:
    prune(tree.fb,mingain)
    
  # If both the subbranches are now leaves, see if they
  # should merged
  if tree.tb.results!=None and tree.fb.results!=None:
    # Build a combined dataset
    tb,fb=[],[]
    for v,c in tree.tb.results.items():
      tb+=[[v]]*c
    for v,c in tree.fb.results.items():
      fb+=[[v]]*c
    
    # Test the reduction in entropy
    delta=entropy(tb+fb)-(entropy(tb)+entropy(fb)/2)

    if delta<mingain:
      # Merge the branches
      tree.tb,tree.fb=None,None
      tree.results=uniquecounts(tb+fb)
  
#apply pruning with a minimal information gain as a parameter
prune(beer_tree,0.95)
drawtree(beer_tree,"beer_tree_pruned.jpg")


--------------------------------------------------------------------------------------------------------------
###  Pruned tree

![Pruned tree](/files/beer_tree_pruned.jpg)

> Why are we not checking minimal gain when building the tree?

--------------------------------------------------------------------------------------------------------------

### Classifying new data

The point is to predict new data

In [109]:
#This function tells us the outcome and the node where the observation lands
def classify(observation,tree):
  if tree.results!=None:
    return tree.results
  else:
    v=observation[tree.col]
    branch=None
    if isinstance(v,int) or isinstance(v,float):
      if v>=tree.value: branch=tree.tb
      else: branch=tree.fb
    else:
      if v==tree.value: branch=tree.tb
      else: branch=tree.fb
    return classify(observation,branch)



classify(['augustiner',6,'dark'],beer_tree)

{'no': 4, 'yes': 1}

In [110]:
classify(['paulaner',3,'light'],beer_tree)

{'no': 5}

In [111]:
classify(['paulaner',2,'light'],beer_tree)

{'yes': 5}

In [112]:
classify(['augustiner',4,'light'],beer_tree)

{'no': 3}

--------------------------------------------------------------------------------------------------------------
## 4. What have we learned

 + Decision trees use information gain to decide where to split
 + Decision trees are easy to run and code without external libraries
 + Don't make the tree too specific to the sample data you are training the model on
 + Pruning helps you improve your predictive performance on new data
 --------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------
## 5. Coming up next

One of the following topics is available:

* Evaluation of Classifiers ---> Not just running, also successfuly running
* Random Forests ---> An ensamble method for Decision Trees, taking care of overfitting
* Matrix Factorization --> Simple but powerful numerical algortithm for dimension reduction 
* Support Vector Machines ---> Separating non-linear data with a linear method using a dimensionality blow-up

--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------

## 6. Further reading and code

[Python Scikit-Learn for trees](http://scikit-learn.org/stable/modules/tree.html)   
[Spark MLib docs for trees](https://spark.apache.org/docs/1.1.1/mllib-decision-tree.html)    
[Overfitting on Wikipedia](https://en.wikipedia.org/wiki/Overfitting)   
[More on overfitting](http://www3.nd.edu/~rjohns15/cse40647.sp14/www/content/lectures/24%20-%20Decision%20Trees%203.pdf)  

--------------------------------------------------------------------------------------------------------------